In [1]:
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core import StorageContext
from llama_index.llms.bedrock import Bedrock
from llama_index.core import Settings
from llama_index.core import PropertyGraphIndex

from llama_index.core.prompts.base import (
    PromptTemplate,
    PromptType,
)

from llama_index.core.indices.property_graph import (
    LLMSynonymRetriever,
    VectorContextRetriever,
)

from llama_index.core.indices.property_graph import (
    ImplicitPathExtractor,
    SimpleLLMPathExtractor,
)

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.embeddings.bedrock import BedrockEmbedding
import json
import os

In [2]:
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="prahar@156",
    url="bolt://localhost:7687",
    database="neo4j"  # Replace with your database name if different
)

In [3]:
# Initialize Bedrock LLM
# Initialize the Bedrock LLM
llm = Bedrock(
    model="anthropic.claude-3-haiku-20240307-v1:0",  # Specify the desired model
    region_name="us-east-1"          # e.g., "us-east-1"
)
# Settings.llm = llm

# # llm = OpenAI(model="gpt-3.5-turbo", temperature=0.3)

embed_model = BedrockEmbedding(
    region_name="us-east-1",  # Replace with your AWS region
    model_name="amazon.titan-embed-text-v2:0"  # Specify the Titan embedding model
)

In [4]:
index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    embed_model=embed_model,
    kg_extractors=[
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            llm=llm,
            num_workers=4,
            max_paths_per_chunk=10,
        ),
    ],
    show_progress=True,
)

In [6]:
kg_retriever = VectorContextRetriever(
    index.property_graph_store,
    embed_model=embed_model,
    similarity_top_k=2,
    path_depth=1,
    # include_text=False,
    include_text=True,
)

nodes = kg_retriever.retrieve(
    "Adani Enterprise Limited?"
)
# nodes = kg_retriever.retrieve('san francisco')
print(len(nodes))
for idx, node in enumerate(nodes):
    print(f">> IDX: {idx}, {node.get_content()}")

0
